In [2]:
import pandas as pd
import numpy as np
train_set = pd.read_csv("training_set.csv")
#feature_matrix = pd.read_csv("feature_matrix.csv")
test_set = pd.read_csv("testing_set.csv")

In [ ]:
#create dictionary to map anime_id to its genre vector
#feature_matrix_indexed = feature_matrix.set_index('anime_id')
#anime_vector = feature_matrix_indexed['genre_vector'].to_dict()
#for key in anime_vector.keys():
    #anime_vector[key] = eval(anime_vector[key])
#print(anime_vector)

In [10]:
source_table = {"train" : train_set,"test" : test_set}
def getRatingVectorOfNthUser(n:int,source):
    #get the rating vector of n-th user and the min and max anime_id
    column_n = str(n) 
    rating = source_table[source][[column_n]].values
    not_minus1_sum,count = 0,0
    for i in rating:
        if i[0] != -1 and i[0] != 0:
            not_minus1_sum += i[0]
            count += 1
    if count != 0:
        mean = round(not_minus1_sum/count,2)
        for i in range(len(rating)):
            if rating[i][0] == -1:
                rating[i][0] = mean
    else:
        for i in range(len(rating)):
            if rating[i][0] == -1:
                rating[i][0] = 5
        
    #normalize the rating value
    del not_minus1_sum,count
    #return 70% of rating vector if source is test else 100%
    
    return rating

print(getRatingVectorOfNthUser(1,"train").shape)
print(getRatingVectorOfNthUser(37015,"test"))
print(getRatingVectorOfNthUser(37020,"test"))

(9724, 1)
[[5.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
[[10.]
 [ 8.]
 [10.]
 ...
 [ 0.]
 [ 0.]
 [ 0.]]


In [11]:
#u1 and u2 must be array
def similarity(u1,u2) -> float:
    
    
    return ((u1.T).dot(u2))/((np.linalg.norm(u1))*np.linalg.norm(u2))
u1 = getRatingVectorOfNthUser(1,"train")
u2 = getRatingVectorOfNthUser(37020,"test")
print(similarity(u1,u2)[0][0])

0.06973723120393556


In [ ]:
#calculate the similarity matrix of all users in our dataset
#this is not necessary for the prediction but  can be useful to cluster the users into distinct groups
similar_matrix = [[0 for i in range(1000)] for j in range(1000)]
print(np.array(similar_matrix))
for i in range(len(similar_matrix)):
    for j in range(len(similar_matrix)):
        if i == j:
            similar_matrix[i][j] = 1
        if similar_matrix[j][i] != 0:
            similar_matrix[i][j] = similar_matrix[j][i]
        else:
            similar_matrix[i][j] = similarity(i+1,j+1)[0][0]
print(np.array(similar_matrix))

In [12]:
test_user_id = test_set.columns.values[1:]
for i in range(len(test_user_id)):
    test_user_id[i] = int(test_user_id[i])
print("ID of users in the testing set is:")
print(*test_user_id)

ID of users in the testing set is:
37015 37020 37032 37034 37040 37049 37052 37074 37097 37100 37107 37110 37117 37133 37149 37163 37196 37199 37219 37229 37233 37236 37237 37264 37267 37270 37326 37327 37345 37372 37383 37386 37399 37405 37411 37423 37425 37454 37459 37464 37490 37494 37502 37508 37520 37524 37549 37558 37576 37599 37609 37621 37641 37662 37674 37698 37702 37720 37725 37754 37781 37800 37803 37806 37808 37823 37873 37874 37903 37916 37922 37928 37965 37971 37979 37985 37998 38004 38009 38042 38045 38079 38083 38087 38100 38120 38137 38152 38162 38213 38233 38237 38253 38263 38275 38278 38290 38312 38318 38322 38324 38367 38381 38383 38388 38433 38436 38462 38467 38471 38487 38494 38515 38521 38525 38531 38537 38541 38544 38555 38557 38582 38586 38599 38622 38644 38732 38747 38753 38757 38769 38777 38790 38797 38803 38813 38820 38833 38852 38866 38873 38883 38885 38895 38898 38899 38925 38951 38954 38956 39001 39006 39028 39049 39072 39103 39135 39152 39155 39182 39194

In [18]:
print(getRatingVectorOfNthUser(1,"train"))
print(getRatingVectorOfNthUser(5,"train"))
print(getRatingVectorOfNthUser(37015,"test"))

[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [8.]
 ...
 [0.]
 [0.]
 [0.]]
[[5.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]


In [20]:
def all_zero(arr):
    for i in arr:
        if i != 0:
            return False
    return True
print(all_zero(getRatingVectorOfNthUser(10,"train")))
print(all_zero(getRatingVectorOfNthUser(5,"train")))
print(all_zero(getRatingVectorOfNthUser(37020,"test")))


False
False


: 

In [ ]:
test_user = None
user_vector = getRatingVectorOfNthUser(test_user)
list_of_similarity = []
max_sim = 0
if all_zero(user_vector):
    #recommend the most popular item if user has no rated items yet.
    continue
else:
    for train_user in dataset:
        train_vector = getRatingVectorOfNthUser(train_user)
        if all_zero(train_vector):
            continue
        else:
            if similarity(user_vector, train_vector)[0][0] > max_sim:
                max_sim = similarity(user_vector, train_vector)[0][0]
                list_of_similarity.append(train_user,max_sim)

lấy ~1000 users mới để làm testing set : done
hàm getRatingVectorOfNthUser(user:int) nên đổi thành  hàm getRatingVectorOfUser(user:int, source = "test" or source = "train") để phân biệt. : done 